In [1]:
from typing import TYPE_CHECKING


if TYPE_CHECKING:
    from math_rag.application.containers import ApplicationContainer
    from math_rag.infrastructure.containers import InfrastructureContainer

    application_container: ApplicationContainer
    infrastructure_container: InfrastructureContainer

In [2]:
RESET = False
%load_ext hooks.notebook_hook

2025-06-26 19:57:58,599 - INFO - datasets - config.py:54 - PyTorch version 2.6.0 available.


In [15]:
hpc_client = infrastructure_container.hpc_client()
pbs_pro_client = infrastructure_container.pbs_pro_client()

In [ ]:
gpu_stats = await hpc_client.gpu_statistics()

In [12]:
for entry in gpu_stats.entries:
    print(f'job_id={entry.job_id}')

    for sub_entry in entry.sub_entries:
        print(f'    {sub_entry}')

job_id=561531
    node='x8000c2s4b0n1' gpu='GPU-2' used_percent=98 mem_used=34260123648
    node='x8000c2s4b0n1' gpu='GPU-0' used_percent=98 mem_used=34260123648


In [13]:
job_stats = await hpc_client.job_statistics()

2025-06-26 15:51:50,766 - INFO - math_rag.infrastructure.clients.ssh_client - ssh_client.py:38 - Command `jobstat -u lpanic | awk 'NR>=4 {print $1, $2, $3, $4, $5, $6, $7, $8}'` in `run` returned stdout: 


In [14]:
job_stats

HPCJobStatistics(entries=[])

In [ ]:
pbs_pro_job_full = await pbs_pro_client.queue_status_full('561538')

In [21]:
pbs_pro_job_full

PBSProJobFull(id='561538.x3000c0s25b0n0.hsn.hpc.srce.hr', name='STDIN', owner='lpanic@x3000c0s27b0n0.hsn.hpc.srce.hr', state=<PBSProJobState.RUNNING: 'R'>, queue=<HPCQueue.CPU_SINGLE: 'cpu-single'>, server='x3000c0s25b0n0.hsn.hpc.srce.hr', checkpoint='u', exec_host='x8000c1s3b1n1/14', exec_vnode='(x8000c1s3b1n1:ncpus=1:mem=1048576kb)', error_path='/dev/pts/0', output_path='/dev/pts/0', dir=PosixPath('/lustre/home/lpanic'), hold_types=<PBSProHoldType.NONE: 'n'>, join_path=<PBSProJoinPath.NONE: 'n'>, keep_files=<PBSProKeepFiles.STDOUT_STDERR_DIRECT: 'oed'>, mail_points=<PBSProMailPoints.ABORT: 'a'>, substate=42, priority=0, session_id='473314', rerunable=False, run_count=1, submit_arguments='-koed -I -l select=1:ncpus=1:mem=1gb -l walltime=01:00:00', project='_pbs_project_default', submit_host='x3000c0s27b0n0.hsn.hpc.srce.hr', created=datetime.datetime(2025, 6, 26, 17, 58, 46), queued=datetime.datetime(2025, 6, 26, 17, 58, 46), modified=datetime.datetime(2025, 6, 26, 18, 3, 58), started=

In [ ]:
pushgateway_client = infrastructure_container.pushgateway_client()
prometheus_hpc_admin_client = infrastructure_container.prometheus_hpc_admin_client()

pushgateway_base_url = infrastructure_container.config.pushgateway.base_url.provided()

In [5]:
PUSHGATEWAY_JOB = 'pbs_monitor'
job_id = '561538'

In [ ]:
import asyncio

from prometheus_client import CollectorRegistry, Gauge, push_to_gateway


registry = CollectorRegistry()
cpu_percent_gauge = Gauge(
    'pbs_job_cpu_percent', 'CPU percentage of PBS job', ['job_id'], registry=registry
)
cpu_time_gauge = Gauge(
    'pbs_job_cpu_seconds_total',
    'Total CPU time of PBS job in seconds',
    ['job_id'],
    registry=registry,
)
num_cpus_gauge = Gauge(
    'pbs_job_num_cpus', 'Number of CPUs allocated to PBS job', ['job_id'], registry=registry
)
mem_bytes_gauge = Gauge(
    'pbs_job_mem_bytes', 'Resident memory bytes of PBS job', ['job_id'], registry=registry
)
vmem_bytes_gauge = Gauge(
    'pbs_job_vmem_bytes', 'Virtual memory bytes of PBS job', ['job_id'], registry=registry
)
wall_time_gauge = Gauge(
    'pbs_job_wall_seconds_total',
    'Wall clock time of PBS job in seconds',
    ['job_id'],
    registry=registry,
)


async def push_resources():
    while True:
        pbs_pro_job_full = await pbs_pro_client.queue_status_full(job_id)
        ru = pbs_pro_job_full.resources_used

        cpu_percent_gauge.labels(job_id=job_id).set(ru.cpu_percent)
        cpu_time_gauge.labels(job_id=job_id).set(ru.cpu_time.total_seconds())
        num_cpus_gauge.labels(job_id=job_id).set(ru.num_cpus)
        mem_bytes_gauge.labels(job_id=job_id).set(ru.mem)
        vmem_bytes_gauge.labels(job_id=job_id).set(ru.vmem)
        wall_time_gauge.labels(job_id=job_id).set(ru.wall_time.total_seconds())

        push_to_gateway(pushgateway_base_url, job=job_id, registry=registry)
        await asyncio.sleep(5)

In [2]:
await push_loop()

CancelledError: 

In [8]:
await pushgateway_client.delete_job(PUSHGATEWAY_JOB)

matchers = [f'pbs_job_cpu_percent{{job_id="{job_id}"}}', f'pbs_job_mem_bytes{{job_id="{job_id}"}}']
await prometheus_hpc_admin_client.delete_series(matchers)